<hr>

<table><tr><td bgcolor=#000000>
<p align='left'>
<font size=6 color=white><b>Before:</b></font><br><br>
</p>
<img src="https://sias.uestc.edu.cn/images/logo.png" align=center hspace=250><br><br>

<font face=宋体 color=white size=5><center><b>
    本项目由电子科大深圳高研院-机器学习课程军事飞机识别课题组所有。<br/>
    欢迎交流指正！</font>
    
<font face=TimesNewRoman size=2 color=lightblue><center><b>
    This project is base on UESTC-SIAS ML Military-Detect Research Group.<br/>
    Welcome communicate with us and correct our mistakes!
    
</td></tr></table>
    
<hr>

<font color=red size=4>  
**Attention: before runnnig these code, change seeting->Accelerator->GPU**

    
**If you need cuda version 11.0, change seeting->Environment->Always use latest environment**
</font>

# Check Environment

In [ ]:
import pycuda.autoinit 
import pycuda.driver as cuda 

(free,total)=cuda.mem_get_info() 
print("Global memory occupancy:%f%% free"%(free*100/total)) 

for devicenum in range(cuda.Device.count()): 
    device=cuda.Device(devicenum) 
    attrs=device.get_attributes() 

    #Beyond this point is just pretty printing 
    print("\n===Attributes for device %d: "%devicenum + str(device.name())) 
    for (key,value) in attrs.items(): 
        print(str(key)+':'+f"\033[1;31m{str(value)}\033[0m")

In [ ]:
!nvidia-smi

In [ ]:
!lsb_release -a

In [ ]:
!nvcc -h

# Install TensorRT

In [ ]:
# install basic packages
!apt-get update && \
    apt-get install -y \
        libnvonnxparsers8=8.0.3-1+cuda11.3 \
        libnvonnxparsers-dev=8.0.3-1+cuda11.3 \
        libnvparsers8=8.0.3-1+cuda11.3\
        libnvparsers-dev=8.0.3-1+cuda11.3 \
        libnvinfer8=8.0.3-1+cuda11.3\
        libnvinfer-plugin8=8.0.3-1+cuda11.3\
        libnvinfer-plugin-dev=8.0.3-1+cuda11.3 \
        libnvinfer-dev=8.0.3-1+cuda11.3 \
        python3-libnvinfer=8.0.3-1+cuda11.3 \
        python3-libnvinfer-dev=8.0.3-1+cuda11.3

In [ ]:
# unzip deb file and install 
!dpkg -i ../input/tensorrt/tensorrt.deb && \
    apt-key add /var/nv-tensorrt-repo-ubuntu1804-cuda11.3-trt8.0.3.4-ga-20210831/7fa2af80.pub && \
    apt-get update && \
    apt-get install -y tensorrt # uff-converter-tf

In [ ]:
# install uff and onnx 
!apt-get install uff-converter-tf -y
!apt-get install onnx-graphsurgeon

In [ ]:
# view tensorrt-pkgs install condition
!dpkg -l | grep TensorRT

In [ ]:
# install/update basic repo
# !python3 -m pip install --upgrade setuptools pip
!python3 -m pip install nvidia-pyindex

In [ ]:
# install tensorrt(python3)
!python3 -m pip install nvidia-tensorrt==8.0.3.4
!python3 -m pip install uff graphsurgeon

# Validate Python Import

In [ ]:
import tensorrt as trt
import ctypes
import uff
import graphsurgeon as gs
import argparse
import tensorflow as tf

print(trt.__version__)
assert trt.Builder(trt.Logger())

# Install Tensorrtx and yolov5

In [ ]:
%cd /kaggle/working
!git clone https://github.com/wang-xinyu/tensorrtx.git
!git clone -b v6.0 https://github.com/ultralytics/yolov5.git
!pip install -r ./yolov5/requirements.txt
!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt

In [ ]:
# solve "ImportError: /usr/lib/x86_64-linux-gnu/libstdc++.so.6: version `GLIBCXX_3.4.26‘ not found"
!cp /opt/conda/pkgs/libstdcxx-ng-11.2.0-he4da1e4_8/lib/libstdc++.so.6.0.29 /usr/lib/x86_64-linux-gnu/
!rm /usr/lib/x86_64-linux-gnu/libstdc++.so.6
!ln -s /usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.29 /usr/lib/x86_64-linux-gnu/libstdc++.so.6
!strings /opt/conda/pkgs/libstdcxx-ng-11.2.0-he4da1e4_8/lib/libstdc++.so.6.0.29 | grep GLIBCXX

# Convert weights and detect

### Generate .wts from pytorch with .pt

In [ ]:
''' 
clone code according to above Different versions of yolov5
download https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt
'''
%cd /kaggle/working
!cp /kaggle/input/tensorrt/gen_wts.py  ./yolov5
!cp /kaggle/input/weights/exp/weights/best.pt ./yolov5
!cp ./yolov5s.pt ./yolov5

'''
a file 'yolov5s.wts' will be generated.
'''
%cd ./yolov5
!python ./gen_wts.py -w best.pt -of yolov5s.pt -o ../best.wts
!cp ../best.wts ../tensorrtx/yolov5

### Build tensorrtx/yolov5 and create Engine

In [ ]:
%cd /kaggle/working
# # // update CLASS_NUM in yololayer.h if your model is trained on custom dataset
!rm -rf ./tensorrtx/yolov5/build/
!mkdir -p ./tensorrtx/yolov5/build
!cp /kaggle/input/tensorrt/yololayer.h /kaggle/working/tensorrtx/yolov5/
!cp /kaggle/input/tensorrt/yolov5.cpp /kaggle/working/tensorrtx/yolov5/

%cd ./tensorrtx/yolov5/build
!cmake ..
!make

In [ ]:
%cd /kaggle/working/tensorrtx/yolov5/build
!apt-get install sudo
!sudo ./yolov5 -s ../best.wts /kaggle/working/best.engine s

### Download video

In [ ]:
!pip install yt-dlp
%cd /kaggle/working
!yt-dlp -o "001.mp4" -f "bestvideo[ext=mp4]" "https://www.youtube.com/watch?v=EgjWQK3QmWg"
# https://www.youtube.com/watch?v=Drl4dPVgX70

### Detect Video and output result

In [ ]:
"""
An example that uses TensorRT's Python api to make inferences.
"""
import ctypes
import os
import shutil
import random
import sys
import threading
import cv2
import re
import time
import torch
import numpy as np
import pycuda.autoinit
import pycuda.driver as cuda
import tensorrt as trt
from pathlib import Path
from threading import Thread


class LoadStreams:  # multiple IP or RTSP cameras
    def __init__(self, source='streams.txt', img_size=640, stride=32, auto=True):
        self.mode = 'stream'
        self.img_size = img_size
        self.stride = stride
        self.count = 0
        self.img_stack, self.imgs, self.fps, self.frames, self.threads = [], None, 0, 0, None
        # clean source names for later      清理源文件名称中不支持的字符
        self.source = re.sub(pattern="[|@#!¡·$€%&()=?¿^*;:,¨´><+]", repl="_", string=source)
        self.auto = auto
        # Start thread to read frames from video stream     启动线程从视频流读取帧
        print(f'{source}... ', end='')
        # i.e. s = '0' local webcam       选择视频流来源是本地摄像头或者输入文件
        s = eval(source) if source.isnumeric() else source
        # try read frame    尝试读取视频流
        cap = cv2.VideoCapture(s)
        assert cap.isOpened(), f'Failed to open {s}'
        # read w/h/fps/frames in video     读取视频长/宽/帧率/帧数信息
        self.width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        self.height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        # 30 FPS fallback   帧率限幅
        self.fps = max(cap.get(cv2.CAP_PROP_FPS) % 100, 0) or 30.0
        # infinite stream fallback    帧数限幅
        self.frames = max(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)), 0) or float('inf')

        _, self.imgs = cap.read()  # guarantee first frame  给imgs传入第一帧，保证开始可以正常读取
#         self.img_stack.append(self.imgs)
        # Start a separate thread to read the video stream data     启动单独线程读取视频流数据
        self.threads = Thread(target=self.update, args=([cap]), daemon=True)
        print(f" success ({self.frames} frames {self.width}x{self.height} at {self.fps:.2f} FPS)")
        self.threads.start()
        print('')  # newline

    def update(self, cap):
        # Read stream frames in daemon thread
        n, f, read = 0, self.frames, 1  # frame number, frame array, inference every 'read' frame
        while cap.isOpened() and n < f:
            n += 1
            # _, self.imgs[index] = cap.read()
            cap.grab()  # 从视频文件或捕获设备获取下一帧
            if n % read == 0:
                success, im = cap.retrieve()  # 解码并返回抓取了的视频帧
                if success:
                    self.imgs = im
                    self.img_stack.append(self.imgs)
                else:
                    self.imgs = self.imgs * 0
            time.sleep(1 / self.fps)  # wait time

    def __iter__(self):
        #         self.count = -1
        return self

    def __next__(self):
        self.count += 1
        while len(self.img_stack) == 0:
            time.sleep(1 / self.fps)
        img0 = self.img_stack.pop(0)
        img = np.array(img0)
        return img

    def __len__(self):
        return len(self.source)  # 1E12 frames = 32 streams at 30 FPS for 30 years

    
CONF_THRESH = 0.5
IOU_THRESHOLD = 0.4

def time_sync():
    # pytorch-accurate time
    if torch.cuda.is_available():
        torch.cuda.synchronize()
    return time.time()

def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    """
    description: Plots one bounding box on image img,
                 this function comes from YoLov5 project.
    param:
        x:      a box likes [x1,y1,x2,y2]
        img:    a opencv image object
        color:  color to draw rectangle, such as (0,255,0)
        label:  str
        line_thickness: int
    return:
        no return

    """
    tl = (
            line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1
    )  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(
            img,
            label,
            (c1[0], c1[1] - 2),
            0,
            tl / 3,
            [225, 255, 255],
            thickness=tf,
            lineType=cv2.LINE_AA,
        )


class YoLov5TRT(object):
    """
    description: A YOLOv5 class that warps TensorRT ops, preprocess and postprocess ops.
    """

    def __init__(self, engine_file_path):
        self.padding_size = {}

        # Create a Context on this device.
        # Attention: Here Cuda's context is different with below tensorrt context!!!!
        # 创建Cuda的context。它非常重要，它作为一个容器，管理了所有对象的生命周期。
        # 注意:这里Cuda的Context与下面tensorrt的Context是不同的
        self.ctx = cuda.Device(0).make_context()
        # Manage the concurrency of execution units.    管理执行单元的并发性
        # CUDA流表示一个GPU操作队列，该队列中的操作将以添加到流中的先后顺序而依次执行。
        # 可以将一个流看做是GPU上的一个任务，不同任务可以并行执行。
        stream = cuda.Stream()
        # Create Tensort engine.    创建Tensorrt引擎
        TRT_LOGGER = trt.Logger(trt.Logger.INFO)
        self.runtime = trt.Runtime(TRT_LOGGER)

        # Deserialize the engine from file.  对文件中的引擎进行反序列化，换句话说，就是解析本地engine文件。
        with open(engine_file_path, "rb") as f:
            engine = self.runtime.deserialize_cuda_engine(f.read())
        # Create an IExecutionContext.
        # 创建一个Engine的Context。Context是真正推理的入口，其储存了中间值，一个Engine允许多个Context。
        context = engine.create_execution_context()

        host_inputs = []
        cuda_inputs = []
        host_outputs = []
        cuda_outputs = []
        bindings = []

        # choose bindings from engine, each binding handle one batch
        # 选取引擎的各个端口，每个端口处理一个batch。端口的具体数值其实是内存地址，即 int(buffer)
        for binding in engine:
            # Get the shape of a binding.   得到端口的传输特征。
            print('bingding:', binding, engine.get_binding_shape(binding))
            # Computes the volume of an iterable.   计算可迭代对象的总容量。
            size = trt.volume(engine.get_binding_shape(binding)) * engine.max_batch_size
            # Returns the numpy-equivalent of a TensorRT DataType. 返回TensorRT数据类型的numpy等效值。
            dtype = trt.nptype(engine.get_binding_dtype(binding))

            # Allocate host and device buffers.     分配主机缓冲区和GPU缓冲区
            # CPU data allocations are pageable by default. The GPU cannot access data directly from pageable host memory,
            # so when a data transfer from pageable host memory to device memory is invoked,
            # the CUDA driver must first allocate a temporary page-locked, or “pinned”, host array,
            # copy the data to the pinned array, and then transfer the data from the pinned array to device memory.
            # 默认情况下，CPU数据分配是可分页的，GPU不能直接从可分页内存(host memory)访问数据。
            # 因此当从可分页内存到显存(device memory)的数据传输被调用时，CUDA驱动程序必须首先分配一个临时的页面锁定，
            # 或“固定”主机阵列，用来将内存数据复制到固定的数组，然后再从固定数组转移到显存。
            host_mem = cuda.pagelocked_empty(size, dtype)
            cuda_mem = cuda.mem_alloc(host_mem.nbytes)

            # Append the device buffer to device bindings.      将GPU缓冲区附加到GPU端口。
            bindings.append(int(cuda_mem))
            # Append to the appropriate list.       将缓冲区添加到合适的缓冲队列。
            if engine.binding_is_input(binding):
                self.input_w = engine.get_binding_shape(binding)[-1]
                self.input_h = engine.get_binding_shape(binding)[-2]
                host_inputs.append(host_mem)
                cuda_inputs.append(cuda_mem)
            else:
                host_outputs.append(host_mem)
                cuda_outputs.append(cuda_mem)

        # Store
        self.stream = stream
        self.context = context
        self.engine = engine
        self.host_inputs = host_inputs
        self.cuda_inputs = cuda_inputs
        self.host_outputs = host_outputs
        self.cuda_outputs = cuda_outputs
        self.bindings = bindings
        self.batch_size = engine.max_batch_size


    def infer(self, raw_image_generator, categories, format='mp4v'):
        # save_name     保存地址
        parent, filename = os.path.split(raw_image_generator.source)
        save_path = f"{time.strftime('%H_%M_%S__')}{filename}"
        # Create video writer   创建视频写入对象
        self.vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*format),
                                          raw_image_generator.fps, 
                                          (raw_image_generator.width, raw_image_generator.height))
        # calculate padding size    计算填充尺寸
        self.Video_padding_size(raw_image_generator.width, raw_image_generator.height)
        print(raw_image_generator.width, raw_image_generator.height, '\n', self.input_w, self.input_h)
        self.categories = categories

        # Set batch     设置batch
        process_num = raw_image_generator.frames // self.batch_size
        print(self.batch_size * process_num, process_num)
        for s in range(process_num):
            # Make self the active context, pushing it on top of the context stack.
            # 使self成为活动context，将它推到Cuda处理context堆栈的顶部。
            self.ctx.push()
            # Restore   恢复
            stream = self.stream
            context = self.context
            engine = self.engine
            host_inputs = self.host_inputs
            cuda_inputs = self.cuda_inputs
            host_outputs = self.host_outputs
            cuda_outputs = self.cuda_outputs
            bindings = self.bindings
            # Do image preprocess   进行图像处理
            batch_image_raw = []
            batch_origin_h = []
            batch_origin_w = []
            batch_input_image = np.empty(shape=[self.batch_size, 3, self.input_h, self.input_w])
            for i in range(self.batch_size):
                image_raw = next(raw_image_generator)
                input_image, image_raw, origin_h, origin_w = self.preprocess_image(image_raw)
                batch_image_raw.append(image_raw)
                batch_origin_h.append(origin_h)
                batch_origin_w.append(origin_w)
                np.copyto(batch_input_image[i], input_image)
            batch_input_image = np.ascontiguousarray(batch_input_image)
            print('frame_count:', raw_image_generator.count, end=', ')

            # Copy input image to host buffer       复制输入图像到内存
            np.copyto(host_inputs[0], batch_input_image.ravel())
            start = time_sync()
            # Transfer input data to the GPU.      将输入数据传输到GPU
            cuda.memcpy_htod_async(cuda_inputs[0], host_inputs[0], stream)
            # Run inference.        进行推理
            context.execute_async(batch_size=self.batch_size, bindings=bindings, stream_handle=stream.handle)
            # Transfer predictions back from the GPU.       从GPU传回预测结果
            cuda.memcpy_dtoh_async(host_outputs[0], cuda_outputs[0], stream)
            # Synchronize the stream        同步流
#             stream.synchronize()
            end = time_sync()
            # Remove any context from the top of the context stack, deactivating it.
            # 从Cuda context堆栈的顶部移除所有context，使其失效。
            self.ctx.pop()
            # Here we use the first row of output in that batch_size = 1
            output = host_outputs[0]
            batch_time = (end - start) * 1000
            # Do postprocess        进行后处理（包括框格标定等）
            for i in range(self.batch_size):
                cv2.putText(batch_image_raw[i], f'batch_fps:{str(int(self.batch_size / batch_time * 1000))}', 
                            (20, 50), 0, (raw_image_generator.width + raw_image_generator.height)*0.002/6, 
                            [10, 10, 10], thickness=4, lineType=cv2.LINE_AA)
                result_boxes, result_scores, result_classid = self.post_process(
                    output[i * 6001: (i + 1) * 6001], batch_origin_h[i], batch_origin_w[i]
                )
                # Draw rectangles and labels on the original image
                for j in range(len(result_boxes)):
                    box = result_boxes[j]
                    plot_one_box(box, batch_image_raw[i], 
                                 label="{}:{:.2f}".format(
                                     self.categories[int(result_classid[j])], result_scores[j]),)
                raw = np.array(batch_image_raw[i])
                # raw: HWC!!! Not WHC!!!                  
                self.vid_writer.write(raw)
            print('input->{}, time->{:.2f}ms, per_time->{:.2f}ms, saving into '
                  .format(s, batch_time, batch_time / self.batch_size) + save_path)
        self.vid_writer.release()  # release previous video writer


    def Video_padding_size(self, w, h):
        """
        Here calculate width and height and paddings

        Args:
            w: Input video width
            h: Input video height

        Returns: dict of padding size

        """
        r_w = self.input_w / w
        r_h = self.input_h / h
        if r_h > r_w:
            tw = self.input_w
            th = int(r_w * h)
            tx1 = tx2 = 0
            ty1 = int((self.input_h - th) / 2)
            ty2 = self.input_h - th - ty1
        else:
            tw = int(r_h * w)
            th = self.input_h
            tx1 = int((self.input_w - tw) / 2)
            tx2 = self.input_w - tw - tx1
            ty1 = ty2 = 0

        self.padding_size = {'tw': tw,
                             'th': th,
                             'tx1': tx1,
                             'tx2': tx2,
                             'ty1': ty1,
                             'ty2': ty2}

    def preprocess_image(self, raw_bgr_image):
        """
        description: Convert BGR image to RGB,
                     resize and pad it to target size, normalize to [0,1],
                     transform to NCHW format.
        param:
            input_image_path: str, image path
        return:
            image:  the processed image
            image_raw: the original image
            h: original height
            w: original width
        """
        image_raw = raw_bgr_image
        pad = self.padding_size

        h, w, c = image_raw.shape
        image = cv2.cvtColor(image_raw, cv2.COLOR_BGR2RGB)
        # Resize the image with long side while maintaining ratio
        image = cv2.resize(image, (pad['tw'], pad['th']))
        # Pad the short side with (128,128,128)
        image = cv2.copyMakeBorder(
            image, pad['ty1'], pad['ty2'], pad['tx1'], pad['tx2'],
            cv2.BORDER_CONSTANT, (128, 128, 128)
        )
        image = image.astype(np.float32)
        # Normalize to [0,1]
        image /= 255.0
        # HWC to CHW format:
        image = np.transpose(image, [2, 0, 1])
        # CHW to NCHW format
        image = np.expand_dims(image, axis=0)
        # Convert the image to row-major order, also known as "C order":
        image = np.ascontiguousarray(image)
        return image, image_raw, h, w

    def xywh2xyxy(self, origin_h, origin_w, x):
        """
        description:    Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
        param:
            origin_h:   height of original image
            origin_w:   width of original image
            x:          A boxes numpy, each row is a box [center_x, center_y, w, h]
        return:
            y:          A boxes numpy, each row is a box [x1, y1, x2, y2]
        """
        y = np.zeros_like(x)
        r_w = self.input_w / origin_w
        r_h = self.input_h / origin_h
        if r_h > r_w:
            y[:, 0] = x[:, 0] - x[:, 2] / 2
            y[:, 2] = x[:, 0] + x[:, 2] / 2
            y[:, 1] = x[:, 1] - x[:, 3] / 2 - (self.input_h - r_w * origin_h) / 2
            y[:, 3] = x[:, 1] + x[:, 3] / 2 - (self.input_h - r_w * origin_h) / 2
            y /= r_w
        else:
            y[:, 0] = x[:, 0] - x[:, 2] / 2 - (self.input_w - r_h * origin_w) / 2
            y[:, 2] = x[:, 0] + x[:, 2] / 2 - (self.input_w - r_h * origin_w) / 2
            y[:, 1] = x[:, 1] - x[:, 3] / 2
            y[:, 3] = x[:, 1] + x[:, 3] / 2
            y /= r_h

        return y

    def post_process(self, output, origin_h, origin_w):
        """
        description: postprocess the prediction
        param:
            output:     A numpy likes [num_boxes,cx,cy,w,h,conf,cls_id, cx,cy,w,h,conf,cls_id, ...]
            origin_h:   height of original image
            origin_w:   width of original image
        return:
            result_boxes: finally boxes, a boxes numpy, each row is a box [x1, y1, x2, y2]
            result_scores: finally scores, a numpy, each element is the score correspoing to box
            result_classid: finally classid, a numpy, each element is the classid correspoing to box
        """
        # Get the num of boxes detected
        num = int(output[0])
        # Reshape to a two dimentional ndarray
        pred = np.reshape(output[1:], (-1, 6))[:num, :]
        # Do nms
        boxes = self.non_max_suppression(pred, origin_h, origin_w, conf_thres=CONF_THRESH, nms_thres=IOU_THRESHOLD)
        result_boxes = boxes[:, :4] if len(boxes) else np.array([])
        result_scores = boxes[:, 4] if len(boxes) else np.array([])
        result_classid = boxes[:, 5] if len(boxes) else np.array([])
        return result_boxes, result_scores, result_classid

    def bbox_iou(self, box1, box2, x1y1x2y2=True):
        """
        description: compute the IoU of two bounding boxes
        param:
            box1: A box coordinate (can be (x1, y1, x2, y2) or (x, y, w, h))
            box2: A box coordinate (can be (x1, y1, x2, y2) or (x, y, w, h))
            x1y1x2y2: select the coordinate format
        return:
            iou: computed iou
        """
        if not x1y1x2y2:
            # Transform from center and width to exact coordinates
            b1_x1, b1_x2 = box1[:, 0] - box1[:, 2] / 2, box1[:, 0] + box1[:, 2] / 2
            b1_y1, b1_y2 = box1[:, 1] - box1[:, 3] / 2, box1[:, 1] + box1[:, 3] / 2
            b2_x1, b2_x2 = box2[:, 0] - box2[:, 2] / 2, box2[:, 0] + box2[:, 2] / 2
            b2_y1, b2_y2 = box2[:, 1] - box2[:, 3] / 2, box2[:, 1] + box2[:, 3] / 2
        else:
            # Get the coordinates of bounding boxes
            b1_x1, b1_y1, b1_x2, b1_y2 = box1[:, 0], box1[:, 1], box1[:, 2], box1[:, 3]
            b2_x1, b2_y1, b2_x2, b2_y2 = box2[:, 0], box2[:, 1], box2[:, 2], box2[:, 3]

        # Get the coordinates of the intersection rectangle
        inter_rect_x1 = np.maximum(b1_x1, b2_x1)
        inter_rect_y1 = np.maximum(b1_y1, b2_y1)
        inter_rect_x2 = np.minimum(b1_x2, b2_x2)
        inter_rect_y2 = np.minimum(b1_y2, b2_y2)
        # Intersection area
        inter_area = np.clip(inter_rect_x2 - inter_rect_x1 + 1, 0, None) * \
                     np.clip(inter_rect_y2 - inter_rect_y1 + 1, 0, None)
        # Union Area
        b1_area = (b1_x2 - b1_x1 + 1) * (b1_y2 - b1_y1 + 1)
        b2_area = (b2_x2 - b2_x1 + 1) * (b2_y2 - b2_y1 + 1)

        iou = inter_area / (b1_area + b2_area - inter_area + 1e-16)

        return iou

    def non_max_suppression(self, prediction, origin_h, origin_w, conf_thres=0.5, nms_thres=0.4):
        """
        description: Removes detections with lower object confidence score than 'conf_thres' and performs
        Non-Maximum Suppression to further filter detections.
        param:
            prediction: detections, (x1, y1, x2, y2, conf, cls_id)
            origin_h: original image height
            origin_w: original image width
            conf_thres: a confidence threshold to filter detections
            nms_thres: a iou threshold to filter detections
        return:
            boxes: output after nms with the shape (x1, y1, x2, y2, conf, cls_id)
        """
        # Get the boxes that score > CONF_THRESH
        boxes = prediction[prediction[:, 4] >= conf_thres]
        # Trandform bbox from [center_x, center_y, w, h] to [x1, y1, x2, y2]
        boxes[:, :4] = self.xywh2xyxy(origin_h, origin_w, boxes[:, :4])
        # clip the coordinates
        boxes[:, 0] = np.clip(boxes[:, 0], 0, origin_w - 1)
        boxes[:, 2] = np.clip(boxes[:, 2], 0, origin_w - 1)
        boxes[:, 1] = np.clip(boxes[:, 1], 0, origin_h - 1)
        boxes[:, 3] = np.clip(boxes[:, 3], 0, origin_h - 1)
        # Object confidence
        confs = boxes[:, 4]
        # Sort by the confs
        boxes = boxes[np.argsort(-confs)]
        # Perform non-maximum suppression
        keep_boxes = []
        while boxes.shape[0]:
            large_overlap = self.bbox_iou(np.expand_dims(boxes[0, :4], 0), boxes[:, :4]) > nms_thres
            label_match = boxes[0, -1] == boxes[:, -1]
            # Indices of boxes with lower confidence scores, large IOUs and matching labels
            invalid = large_overlap & label_match
            keep_boxes += [boxes[0]]
            boxes = boxes[~invalid]
        boxes = np.stack(keep_boxes, 0) if len(keep_boxes) else np.array([])
        return boxes

    def destroy(self):
        # Remove any context from the top of the context stack, deactivating it.
        self.ctx.pop()
#         self.ctx.destory()
#         del self.ctx


class inferThread:
    def __init__(self, yolov5_wrapper, src_path, categories):
        #         threading.Thread.__init__(self)
        self.yolov5_wrapper = yolov5_wrapper
        self.src_path = src_path
        self.categories = categories

    def run(self):
        # Load stream       加载视频流
        raw_image_generator = LoadStreams(source=self.src_path)
        self.yolov5_wrapper.infer(raw_image_generator, self.categories)


%cd /kaggle/working
# load custom plugin and engine
PLUGIN_LIBRARY = "./tensorrtx/yolov5/build/libmyplugins.so"
engine_file_path = "./best.engine"
src_file_path = "./001.mp4"

# load labels
categories = ['A10','A400M', 'AG600','B1','B2','B52','Be200', 'C130','C17','C5', 'E2','EF2000',
    'F117','F14','F15','F16','F18','F22','F35','F4',
    'JAS39','MQ9','Mig31','Mirage2000','RQ4','Rafale',
    'SR71','Su57','Tu160','Tu95','U2','US2', 'V22','XB70','YF23','J20']

# load plugins
ctypes.CDLL(PLUGIN_LIBRARY)
# a YoLov5TRT instance
yolov5_wrapper = YoLov5TRT(engine_file_path)

In [ ]:
print('batch size is', yolov5_wrapper.batch_size)
infer = inferThread(yolov5_wrapper, src_file_path, categories)
infer.run()

### Detach resources

In [ ]:
# 实例释放
yolov5_wrapper.ctx.detach() 
# yolov5_wrapper.ctx
del yolov5_wrapper.host_inputs
del yolov5_wrapper.host_outputs
del yolov5_wrapper.cuda_inputs
del yolov5_wrapper.cuda_outputs
del yolov5_wrapper.stream
del yolov5_wrapper.context
del yolov5_wrapper.engine
del yolov5_wrapper.bindings